In [1]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
#Datasets path


# 20 percent
train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/20 percent/Training/'
val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/20 percent/Validation/'

# 10 percent
#train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/10 percent/Training/'
#val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/10 percent/Validation/'



In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 30805 files belonging to 5 classes.


2023-01-12 08:30:47.189420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 08:30:47.238236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 08:30:47.238571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 08:30:47.258082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 8188 files belonging to 5 classes.


In [5]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-12 08:30:58.513102: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-12 08:30:58.940153: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


241/241 [==============================] - ETA: 0s - loss: 1.1482 - accuracy: 0.5149 - sensitivity_at_specificity: 0.9136 - specificity_at_sensitivity: 0.8937 - recall: 0.3026 - precision: 0.6978
Epoch 1: val_accuracy improved from -inf to 0.60992, saving model to ECG_Spectrogram_Model.h5
241/241 [==============================] - 206s 808ms/step - loss: 1.1482 - accuracy: 0.5149 - sensitivity_at_specificity: 0.9136 - specificity_at_sensitivity: 0.8937 - recall: 0.3026 - precision: 0.6978 - val_loss: 0.9467 - val_accuracy: 0.6099 - val_sensitivity_at_specificity: 0.9645 - val_specificity_at_sensitivity: 0.9405 - val_recall: 0.4933 - val_precision: 0.6831
Epoch 2/50
241/241 [==============================] - ETA: 0s - loss: 0.8474 - accuracy: 0.6615 - sensitivity_at_specificity: 0.9793 - specificity_at_sensitivity: 0.9623 - recall: 0.5573 - precision: 0.7355
Epoch 2: val_accuracy improved from 0.60992 to 0.65914, saving model to ECG_Spectrogram_Model.h5
241/241 [========================

Epoch 14/50
241/241 [==============================] - ETA: 0s - loss: 0.5009 - accuracy: 0.8102 - sensitivity_at_specificity: 0.9973 - specificity_at_sensitivity: 0.9937 - recall: 0.7755 - precision: 0.8410
Epoch 14: val_accuracy did not improve from 0.70652
241/241 [==============================] - 176s 732ms/step - loss: 0.5009 - accuracy: 0.8102 - sensitivity_at_specificity: 0.9973 - specificity_at_sensitivity: 0.9937 - recall: 0.7755 - precision: 0.8410 - val_loss: 0.9269 - val_accuracy: 0.6979 - val_sensitivity_at_specificity: 0.9562 - val_specificity_at_sensitivity: 0.9734 - val_recall: 0.6584 - val_precision: 0.7365
Epoch 15/50
241/241 [==============================] - ETA: 0s - loss: 0.4976 - accuracy: 0.8108 - sensitivity_at_specificity: 0.9987 - specificity_at_sensitivity: 0.9937 - recall: 0.7777 - precision: 0.8418
Epoch 15: val_accuracy did not improve from 0.70652
241/241 [==============================] - 176s 732ms/step - loss: 0.4976 - accuracy: 0.8108 - sensitivity_

Epoch 27/50
241/241 [==============================] - ETA: 0s - loss: 0.3924 - accuracy: 0.8512 - sensitivity_at_specificity: 0.9987 - specificity_at_sensitivity: 0.9973 - recall: 0.8290 - precision: 0.8724
Epoch 27: val_accuracy did not improve from 0.70652
241/241 [==============================] - 177s 733ms/step - loss: 0.3924 - accuracy: 0.8512 - sensitivity_at_specificity: 0.9987 - specificity_at_sensitivity: 0.9973 - recall: 0.8290 - precision: 0.8724 - val_loss: 1.2521 - val_accuracy: 0.6981 - val_sensitivity_at_specificity: 0.9248 - val_specificity_at_sensitivity: 0.9711 - val_recall: 0.6761 - val_precision: 0.7170
Epoch 28/50
241/241 [==============================] - ETA: 0s - loss: 0.3846 - accuracy: 0.8540 - sensitivity_at_specificity: 0.9987 - specificity_at_sensitivity: 0.9975 - recall: 0.8337 - precision: 0.8755
Epoch 28: val_accuracy did not improve from 0.70652
241/241 [==============================] - 177s 732ms/step - loss: 0.3846 - accuracy: 0.8540 - sensitivity_

Epoch 40/50
241/241 [==============================] - ETA: 0s - loss: 0.3079 - accuracy: 0.8828 - sensitivity_at_specificity: 0.9993 - specificity_at_sensitivity: 0.9989 - recall: 0.8676 - precision: 0.8973
Epoch 40: val_accuracy did not improve from 0.70652
241/241 [==============================] - 177s 732ms/step - loss: 0.3079 - accuracy: 0.8828 - sensitivity_at_specificity: 0.9993 - specificity_at_sensitivity: 0.9989 - recall: 0.8676 - precision: 0.8973 - val_loss: 1.3789 - val_accuracy: 0.7031 - val_sensitivity_at_specificity: 0.9157 - val_specificity_at_sensitivity: 0.9719 - val_recall: 0.6876 - val_precision: 0.7160
Epoch 41/50
241/241 [==============================] - ETA: 0s - loss: 0.3197 - accuracy: 0.8774 - sensitivity_at_specificity: 0.9992 - specificity_at_sensitivity: 0.9988 - recall: 0.8633 - precision: 0.8929
Epoch 41: val_accuracy did not improve from 0.70652
241/241 [==============================] - 176s 732ms/step - loss: 0.3197 - accuracy: 0.8774 - sensitivity_

In [26]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8973087072372437
Validation Accuracy:  0.7289777398109436
Validation Specificity:  0.9811514019966125
Validation Sensitivity:  0.9823624491691589
Validation Recall:  0.6969320774078369
Validation Precision:  0.7724354267120361
Validation Loss:  0.7525846362113953
